In [1]:
import torch
import torch_radon as tr
import astra
import numpy as np
import time
import matplotlib.pyplot as plt
import odl
import random

def odl_angles(start, end, n_angles):
    apart = odl.uniform_partition(start, end, n_angles)
    
    correction = abs(end - start) / (2 * n_angles)
    mi = start + correction
    ma = end - correction
    angles = np.linspace(mi, ma, n_angles, endpoint=True)
    
    return apart, angles

def random_3d_volume(volume):
    sz, sy, sx = volume.shape()
    cube = np.zeros(volume.shape(), dtype=np.float32)
    for i in range(25):
        x = random.randint(0, sx)
        y = random.randint(0, sy)
        z = random.randint(0, sz)
        dx = random.randint(1, 32)
        dy = random.randint(1, 32)
        dz = random.randint(1, 32)
        w = random.uniform(0, 1)
        cube[max(z-dz, 0):min(z+dz, sz-1), max(y-dy, 0):min(y+dy, sy-1), max(x-dx, 0):min(x+dx, sx-1)] += w
    return cube

dtype = np.float32
device = torch.device("cuda")

apart, angles = odl_angles(-np.pi, np.pi, 256)

volume = tr.Volume3D(256, 1024, 1024, voxel_size=(1.0, 1.0, 1.0), center=(0, 0, 0))
radon = tr.ConeBeam(1024, angles, volume=volume,
                    src_dist=1536, det_dist=1536, 
                    det_spacing_u=2.0, det_spacing_v=2.0, pitch=0.0, base_z=0.0)

shape = volume.shape()[::-1]
space = odl.uniform_discr(volume.min(), volume.max(), shape, dtype=dtype, nodes_on_bdry=True)
dpart = odl.uniform_partition(
    [-radon.det_count_u/2 * radon.det_spacing_u, -radon.det_count_v/2 * radon.det_spacing_v],
    [radon.det_count_u/2 * radon.det_spacing_u, radon.det_count_v/2 * radon.det_spacing_v],
    [radon.det_count_u, radon.det_count_v]
, nodes_on_bdry=False)

geometry = odl.tomo.ConeBeamGeometry(apart, dpart, radon.src_dist, radon.det_dist, pitch=radon.pitch,
                                     offset_along_axis=radon.base_z)
operator = odl.tomo.RayTransform(space, geometry)

cube = random_3d_volume(volume)

proj_data = operator(cube.transpose(2, 1, 0))
# odl_bp = np.asarray(operator.adjoint(proj_data)).transpose(2, 1, 0)
odl_sino = np.asarray(proj_data).transpose(0, 2, 1)

# x = torch.FloatTensor(cube).to(device)
# y = radon.forward(x)
# bp = radon.backward(y)

# odl_bp = odl_bp * torch.norm(bp).item() / np.linalg.norm(odl_bp)


# print(torch.sum(y*y), torch.sum(bp*x))
# print(np.sum(odl_sino*odl_sino), np.sum(odl_bp*cube.transpose(2, 1, 0)))


# angle_id = 0
# fig, ax = plt.subplots(1, 3)
# ax = ax.ravel()
# ax[0].imshow(odl_sino[angle_id])
# ax[0].axis("off")
# ax[1].imshow(y[angle_id].cpu().numpy())
# ax[1].axis("off")
# ax[2].imshow(np.abs(odl_sino[angle_id] - y[angle_id].cpu().numpy()))
# ax[2].axis("off")

# slice_id = 46
# fig, ax = plt.subplots(1, 3)
# ax = ax.ravel()
# ax[0].imshow(odl_bp[slice_id])
# ax[0].axis("off")
# ax[1].imshow(bp[slice_id].cpu().numpy())
# ax[1].axis("off")
# ax[2].imshow(np.abs(odl_bp[slice_id] - bp[slice_id].cpu().numpy()))
# ax[2].axis("off")

# plt.show()

print("Forward", np.linalg.norm(odl_sino - y.cpu().numpy()) / np.linalg.norm(proj_data))
print("Backward", np.linalg.norm(odl_bp - bp.cpu().numpy()) / np.linalg.norm(odl_bp))
print(np.linalg.norm(bp.cpu().numpy()), np.linalg.norm(odl_bp))

/home/matteo/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/matteo/miniconda3/lib/python3.7/site-packages/odl/space/npy_tensors.py:483: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if dtype not in (np.object, np.void):
/home/matteo/miniconda3/lib/python3.7/site-packages/odl/set/domain.py:256: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20

NameError: name 'y' is not defined

In [ ]:
odl.__version__